# Flughafen Projekt: Aufbau und Analyse mit SQL

Dieses Projekt dokumentiert den Aufbau eines Flughafenmanagement-Systems mit PostgreSQL. \
Ich habe die Datenbank erstellt, die Tabellen definiert, die gelieferten Daten importiert und erste Analysen durchgeführt. 

Die Arbeit umfasste drei Hauptbereiche:

1. Erstellung eines Entity-Relationship-Diagramms (ERD) zur Modellierung der Datenbeziehungen.
2. Aufbau der Datenbanktabellen basierend auf dem ERD und Import der vorliegenden Daten.
3. Manipulation der Daten
4. Analyse der Daten mittels SQL-Abfragen

## 1. Erstellung des Entity-Relationship-Diagramms (ERD)

Für die Modellierung der Datenbank habe ich die folgenden Anforderungen berücksichtigt:

- Jeder Passagier erhielt eine eindeutige ID als Primärschlüssel, sowie Vor- und Nachname und Passnummer.
- Zusätzliche Passagierdetails (Geburtstag, Geschlecht, Adresse, Kontaktdaten) wurden in einer separaten Tabelle gespeichert, verknüpft über die Passagier-ID.
- Buchungen enthielten Platz und Preis sowie Verweise auf Passagier und Flug.
- Flüge hatten eine eindeutige ID, Flugnummer, Abflug- und Ankunftszeit, Fluglinie, Flugzeug und Abflug- sowie Ankunftsflughafen.
- Fluglinien hatten eine ID, IATA-Code, Firmenname und Heimflughafen.
- Flugzeuge hatten eine ID, Kapazität, gehörten zu genau einem Flugzeugtyp und einer Fluglinie.
- Flugzeugtypen wurden durch ID, Bezeichnung und Beschreibung definiert.
- Flughäfen hatten eine ID, Name, Stadt, Land und geografische Koordinaten (Breite, Länge).

Auf Basis dieser Anforderungen habe ich ein ERD erstellt, das die Beziehungen zwischen den Entitäten visualisierte.

![Entity Relationship Diagram](./img/ERD_flughafen.png)

## 2. Aufbau der Datenbank

Basierend auf meinem ERD habe ich die Datenbank `projekt_flughafen` erstellt. 
Die Tabellen wurden entsprechend der modellierten Strukturen aufgebaut und anschließend mit den bereitgestellten Datensätzen befüllt.

Die Datenbank umfasste unter anderem die folgenden Tabellen:
- flughafen
- fluglinie
- flugzeug
- flugzeugtyp
- flug
- passagier
- passagierdetails
- buchung


In [ ]:
-- Beispiel: Erstellung der Tabelle "flughafen"
DROP TABLE IF EXISTS flughafen;

CREATE TABLE flughafen (
    flughafen_id INTEGER PRIMARY KEY,
    name VARCHAR(50) NOT NULL,
    stadt VARCHAR(50),
    land VARCHAR(50) NOT NULL,
    breite NUMERIC(11,8) NOT NULL,
    laenge NUMERIC(11,8) NOT NULL
);

Die Daten wurden anschließend importiert, indem ich die bereitgestellten Datensätze mit dem Befehl `INSERT INTO flughafen VALUES (...)` eingefügt habe.\
Derselbe Prozess wurde für alle weiteren Tabellen angewendet.


## 3. Manipulation und Analyse der Daten

Nach der Erstellung und dem Import der Tabellen habe ich folgende Aufgaben durchgeführt:


In [ ]:
-- 1. Benenne die Tabelle `buchung_sample` zu `buchung` um.

ALTER TABLE buchung_sample RENAME TO buchung;


-- 2. Füge eine neue Person in die Tabelle `passagier` ein, wobei die ID automatisch generiert werden soll. Notiere, welche ID der neuen Zeile zugewiesen wurde.

INSERT INTO passagier (passnummer, vorname, nachname)
VALUES ('P1234567', 'Margherita', 'Carbon')
RETURNING *;


-- passagier_id ist 1

SELECT * FROM passagier
ORDER BY passagier_id asc;


-- 3. Füge der Tabelle passagier eine neue Spalte `staatsangehörigkeit` hinzu.

ALTER TABLE passagier ADD COLUMN staatsangehoerigkeit VARCHAR(50) NOT NULL DEFAULT 'Unbekannt';


-- 4. Weise der zuvor erstellten Person eine Nationalität zu.

UPDATE passagier
SET staatsangehoerigkeit = 'Italian'
WHERE passagier_id = 1
RETURNING *;


-- 5. Entferne die Spalte `staatsangehörigkeit` aus der Tabelle `passagier`.

ALTER TABLE passagier DROP COLUMN staatsangehoerigkeit;


## 4. Datenanalyse der Datenbank

Ich habe die Daten der erstellten Datenbank analysiert und dazu Queries formuliert, um die vorgegebenen Übungsaufgaben zu bearbeiten. \
Die Aufgaben reichen von einfachen Aggregationen bis hin zu komplexeren Joins und CTEs.


In [ ]:
-- 1. Ermittlung des minimalen, maximalen und durchschnittlichen Buchungspreises.

SELECT
	min(preis) as min_preis,
	max(preis) as max_preis,
	round(avg(preis),2) as durchschnitts_preis
FROM buchung;


-- 2. Abruf der Namen der Passagiere mit dem höchsten Buchungskosten.

SELECT 
	p.vorname,
	p.nachname,
	b.preis
FROM passagier p
INNER JOIN buchung b USING(passagier_id)
WHERE b.preis = (SELECT
					max(preis) as max_preis
				 FROM buchung);


-- 3. Berechnung der durchschnittlichen teuersten Ticketpreise pro Fluglinie. 

SELECT 
	fl.fluglinie_id,
	fl.firmennamen,
	count(b.buchung_id) as Anzahl_Buchungen,
	sum(b.preis) as Summe_Preis_Buchungen,
	round(sum(b.preis) / count(b.buchung_id),2) as durchschnitts_preis_buchungen
FROM fluglinie fl
INNER JOIN flug f USING(fluglinie_id)
INNER JOIN buchung b USING(flug_id)
GROUP BY fl.fluglinie_id
ORDER BY durchschnitts_preis_buchungen desc;


-- 4. Identifizierung von Flugzeugen mit der höchsten Kapazität, die vom Flughafen ALTAMIRA abgeflogen sind.

SELECT
	ft.bezeichnung,
	fg.kapazitaet,
	fh.name
FROM flugzeug fg
INNER JOIN flugzeug_typ ft USING(typ_id)
INNER JOIN flug f USING(flugzeug_id)
INNER JOIN flughafen fh ON f.von = fh.flughafen_id
WHERE fh.name = 'ALTAMIRA' and fg.kapazitaet > 200;


-- 5. Zählung der Passagiere der Fluglinie Spain Airlines im Zeitraum vom 06.06.2015 bis zum 08.06.2015.

SELECT
	fl.firmennamen,
	count(buchung_id) as Anzahl_passagiere
FROM fluglinie fl
INNER JOIN flug f      USING(fluglinie_id)
INNER JOIN buchung b   USING(flug_id)
INNER JOIN passagier p USING(passagier_id)
WHERE fl.firmennamen = 'Spain Airlines' AND f.ankunft BETWEEN '2015-06-06' AND '2015-06-08'
GROUP BY fl.firmennamen;


-- 6. Darstellung der Flüge mit Flugnummer, Kapazität des Flugzeugs und Buchungsanzahl. Zusätzlich wird angezeigt, ob der Flug über 5 % ausgelastet war.

SELECT
    flug_id,
    flugnr,
    kapazitaet,
    Anzahl_Buchungen,
    CASE 
        WHEN (Anzahl_Buchungen::numeric / kapazitaet) * 100 > 5 THEN 'Ja'
        ELSE 'Nein'
    END AS ausgelastet_über_5_prozent
FROM
    (
        SELECT
            f.flug_id,
            f.flugnr,
            fg.kapazitaet,
            COUNT(b.buchung_id) AS Anzahl_Buchungen
        FROM flug f
        INNER JOIN flugzeug fg USING (flugzeug_id)
        LEFT JOIN buchung b USING (flug_id)
        GROUP BY f.flug_id, f.flugnr, fg.kapazitaet
    ) AS a;


-- 7. Ermittlung der Fluglinien, die am häufigsten zum Flughafen KAGOSHIMA fliegen.

SELECT 
	fl.firmennamen,
	count(f.nach) as Anzahl_fluege
FROM fluglinie fl
INNER JOIN flug f USING(fluglinie_id)
INNER JOIN flughafen fh ON f.nach = fh.flughafen_id
WHERE fh.name = 'KAGOSHIMA'
GROUP BY fl.firmennamen
ORDER BY Anzahl_fluege DESC;


-- 8. Bestimmung der Flugzeuge einer Fluglinie mit italienischem Heimatflughafen, die die meisten Flüge durchgeführt haben, inklusive des Flugzeugtyps.

SELECT
	fg.flugzeug_id,
	fl.firmennamen,
	fh.land,
	count(flug_id) as anzahl_fluege,
	ft.bezeichnung
FROM flughafen fh
INNER JOIN fluglinie fl ON fl.heimat_flughafen = fh.flughafen_id
INNER JOIN flug f USING(fluglinie_id)
INNER JOIN flugzeug fg  USING(flugzeug_id)
INNER JOIN flugzeug_typ ft USING(typ_id)
WHERE fh.land = 'ITALY'
GROUP BY fg.flugzeug_id, fl.firmennamen, fh.land, ft.bezeichnung
ORDER BY anzahl_fluege desc;


-- 9. Berechnung der gesamten Anteile aller Buchungen je nach Flugzeugtyp in Prozent.

WITH a as ( SELECT 
				ft.bezeichnung,
				count(b.buchung_id) as Anzahl_Buchung
			FROM buchung b
			INNER JOIN flug f USING(flug_id)
			INNER JOIN flugzeug fg USING(flugzeug_id)
			INNER JOIN flugzeug_typ ft USING(typ_id)
			GROUP BY ft.bezeichnung),
			
	b as ( SELECT 
				sum(Anzahl_Buchung) as Gesamt
			FROM a)
SELECT 
	a.bezeichnung,
	a.Anzahl_Buchung,
	round(((Anzahl_Buchung/b.Gesamt)*100), 2) as Anteil_Prozent
FROM a
INNER JOIN b ON TRUE
GROUP BY a.bezeichnung, a.Anzahl_Buchung, b.Gesamt
ORDER BY Anteil_Prozent desc;
